In [8]:
# Start writing code here...
# Start writing code here...
# Start writing code here..
!pip install statsmodels
import numpy as np
import pandas as pd 
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from statsmodels.formula.api import ols
import seaborn as sns
import matplotlib.pyplot as plt

You should consider upgrading via the '/opt/venv/bin/python -m pip install --upgrade pip' command.


In [14]:
list = ['jp','kr','us']
data = [0]*3
for i in range(len(list)):
    data[i] = pd.read_csv("/work/"+list[i]+".csv")
    data[i] = data[i][data[i].loc[:,'views']<=400000]
data[1]

,likes,dislikes,views,comment_count,category_id
0,39.19,0.80,927.85,8.98,10.0
1,52.59,2.60,5594.63,7.93,10.0
2,2.28,0.07,336.57,0.20,10.0
3,9.72,0.80,1863.02,2.59,10.0
4,26.44,0.81,3206.34,7.39,10.0
...,...,...,...,...,...
4449,1.94,0.13,466.56,1.28,17.0
4450,5.49,0.25,643.76,2.48,17.0
4451,44.72,2.51,5170.05,25.52,17.0
4452,4.32,0.18,1359.87,1.15,17.0


In [10]:
for i in range(3):
    dummy = data[i].loc[:,"category_id"]
    dummy = pd.get_dummies(dummy,prefix = "id",)
    data[i] = pd.concat([data[i],dummy],axis=1)
    print(data[i].columns)
    data[i] = data[i].rename(columns={'id_10':'id10s','id_15':'id15s','id_17':'id17s'})
    data[i] = data[i].rename(columns={'id_10.0':'id10s','id_15.0':'id15s','id_17.0':'id17s'})

Index(['likes', 'dislikes', 'views', 'comment_count', 'category_id', 'id_10',
       'id_15', 'id_17'],
      dtype='object')
Index(['likes', 'dislikes', 'views', 'comment_count', 'category_id', 'id_10.0',
       'id_15.0', 'id_17.0'],
      dtype='object')
Index(['likes', 'dislikes', 'views', 'comment_count', 'category_id', 'id_10.0',
       'id_15.0', 'id_17.0'],
      dtype='object')


In [12]:
model = [0]*3
for i in range(3):
    model[i] = ols("likes~views+id10s+I(id15s)+id17s+comment_count+dislikes",data = data[i]).fit()
    print(model[i].summary())


                            OLS Regression Results                            
Dep. Variable:                  likes   R-squared:                       0.833
Model:                            OLS   Adj. R-squared:                  0.833
Method:                 Least Squares   F-statistic:                     4425.
Date:                Sat, 19 Dec 2020   Prob (F-statistic):               0.00
Time:                        10:41:49   Log-Likelihood:                -33453.
No. Observations:                4450   AIC:                         6.692e+04
Df Residuals:                    4444   BIC:                         6.696e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         2.2481      5.351      0.420

In [19]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
reg = LinearRegression()
X = pd.DataFrame()
for i in range(3):
    X = data[i].iloc[:,1:]
    y = data[i].iloc[:,0]
    cv_results = cross_val_score(reg,X,y,cv = 5,scoring = 'neg_mean_squared_error')
    print(np.mean(cv_results))
    

-650696.1950435517
-623115.1240326826
-609125.5681057548


In [41]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn import datasets
from sklearn.decomposition import PCA
kr=data[1]
us=data[2]
X =kr.iloc[:,1:4]
y =kr['likes']
pca = PCA(n_components=3)
X_p = pca.fit(X).transform(X)
pca = PCA(n_components=3)
pca.fit(X)

print(pca.explained_variance_ratio_)
print(pca.explained_variance_)
X =us.iloc[:,1:4]
y =us['likes']
pca = PCA(n_components=3)
X_p = pca.fit(X).transform(X)
pca = PCA(n_components=3)
pca.fit(X)
print(pca.explained_variance_ratio_)
print(pca.explained_variance_)

[9.99960806e-01 3.81124951e-05 1.08137994e-06]
[8.46330068e+08 3.22570148e+04 9.15240233e+02]
[9.99974426e-01 2.32133010e-05 2.36025870e-06]
[2.51442700e+09 5.83696434e+04 5.93484997e+03]


In [43]:
final_model_jp=ols('likes~dislikes+views',data=kr).fit()
final_model_us=ols('likes~dislikes+views',data=us).fit()
print(final_model_jp.summary())
print(final_model_us.summary())

In [48]:
X = pd.concat([kr["dislikes"],kr["views"]],axis=1)
y = kr["likes"]
cv_results = cross_val_score(reg,X,y,cv = 5,scoring = 'neg_mean_squared_error')
print(np.mean(cv_results))

-1373419.1222932283


In [49]:
X = pd.concat([us["dislikes"],us["views"]],axis=1)
y = us["likes"]
cv_results = cross_val_score(reg,X,y,cv = 5,scoring = 'neg_mean_squared_error')
print(np.mean(cv_results))

-2264265.459720722
